<a href="https://colab.research.google.com/github/GePajarinen/Studying-Spark/blob/main/BigData_wee2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q --upgrade pip -q
!pip install -q pyspark

import pyspark
from pyspark import SparkFiles, SparkConf
from pyspark.context import SparkContext
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder \
        .appName("Spark 1") \
        .master("local[2]") \
        .getOrCreate()

sc = SparkContext.getOrCreate(SparkConf())

In [3]:
"""
census_df = spark.read \
          .format("csv") \
          .option("header", "true") \
          .option("inferSchema", "true") \
          .load("https://raw.githubusercontent.com/GePajarinen/coursera/master/big-data-2/csv/census.csv")
"""

'\ncensus_df = spark.read           .format("csv")           .option("header", "true")           .option("inferSchema", "true")           .load("https://raw.githubusercontent.com/GePajarinen/coursera/master/big-data-2/csv/census.csv")\n'

In [4]:
url = "https://raw.githubusercontent.com/GePajarinen/coursera/master/big-data-2/csv/census.csv"
sc.addFile(url)

tmp = SparkFiles.get('census.csv')
#print(tmp)

census = spark.read.csv(tmp, header=True, inferSchema= True)
census.show()

+------+------+--------+-----+------+-------+---------------+-------------+-----------------+---------------+---------------+---------------+---------------+---------------+---------------+------------+------------+------------+------------+------------+------------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+--------------+--------------+--------------+--------------+--------------+--------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+---------------+---------------+---------------+---------------+---------------+---------------+----------+----------+----------+----------+----------+----------+------------+------------+------------+------------+------------+------------+-------------------+---------------+---------------+---------------+---------------+---------------+---------------+------------+------------+---

Question 1     
(Questions 1-3 pertain to the video lecture “Exploring the Relational Data Model of CSV”)    
</br>
What is the approximate population of La Paz county in the state of Arizona for the CENSUS2010POP (column H)? (Choose the best answer.):   
15000   
10000   
20000   
25000   

In [5]:
census.createOrReplaceGlobalTempView("census")

In [6]:
spark.sql("SELECT CTYNAME, CENSUS2010POP FROM global_temp.census WHERE CTYNAME LIKE '%La Paz%'").show(100)

+-------------+-------------+
|      CTYNAME|CENSUS2010POP|
+-------------+-------------+
|La Paz County|        20489|
+-------------+-------------+



Question 2   
What county in the state of Wyoming has the smallest estimated population?   

Platte   
Sweetwater   
Uinta   
Niobrara   

In [7]:
spark.sql("SELECT STNAME, CTYNAME, CENSUS2010POP FROM global_temp.census WHERE STNAME = 'Wyoming' ORDER BY CENSUS2010POP ASC ").show(55)

+-------+------------------+-------------+
| STNAME|           CTYNAME|CENSUS2010POP|
+-------+------------------+-------------+
|Wyoming|   Niobrara County|         2484|
|Wyoming|Hot Springs County|         4812|
|Wyoming|      Crook County|         7083|
|Wyoming|     Weston County|         7208|
|Wyoming|   Washakie County|         8533|
|Wyoming|    Johnson County|         8569|
|Wyoming|     Platte County|         8667|
|Wyoming|   Sublette County|        10247|
|Wyoming|   Big Horn County|        11668|
|Wyoming|     Goshen County|        13249|
|Wyoming|   Converse County|        13833|
|Wyoming|     Carbon County|        15885|
|Wyoming|    Lincoln County|        18106|
|Wyoming|      Uinta County|        21118|
|Wyoming|      Teton County|        21294|
|Wyoming|       Park County|        28205|
|Wyoming|   Sheridan County|        29116|
|Wyoming|     Albany County|        36299|
|Wyoming|    Fremont County|        40123|
|Wyoming| Sweetwater County|        43806|
|Wyoming|  

In [8]:
url_sensor = "https://raw.githubusercontent.com/GePajarinen/coursera/master/big-data-2/sensor/wx-data.txt"
sc.addFile(url_sensor)

tmp_sensor = SparkFiles.get('wx-data.txt')
#print(tmp)

sensor = spark.read.text(tmp_sensor, \
                         )
sensor.show(truncate=False)

+----------------------------------------------------------------+
|value                                                           |
+----------------------------------------------------------------+
|1419408000\t0R1,Dn=038D,Dm=079D,Dx=120D,Sn=2.8M,Sm=6.0M,Sx=8.1M |
|1419408001\t0R1,Dn=038D,Dm=074D,Dx=120D,Sn=2.8M,Sm=6.2M,Sx=8.8M |
|1419408002\t0R1,Dn=038D,Dm=071D,Dx=120D,Sn=2.8M,Sm=6.5M,Sx=9.2M |
|1419408003\t0R1,Dn=038D,Dm=067D,Dx=120D,Sn=2.8M,Sm=6.8M,Sx=9.5M |
|1419408004\t0R1,Dn=038D,Dm=062D,Dx=081D,Sn=2.8M,Sm=7.2M,Sx=10.0M|
|1419408005\t0R1,Dn=038D,Dm=060D,Dx=073D,Sn=2.8M,Sm=7.8M,Sx=10.0M|
|1419408006\t0R1,Dn=054D,Dm=064D,Dx=080D,Sn=5.6M,Sm=8.5M,Sx=10.0M|
|1419408006\t0R2,Ta=13.9C,Ua=28.5P,Pa=889.9H                     |
|1419408007\t0R1,Dn=054D,Dm=066D,Dx=080D,Sn=6.8M,Sm=8.7M,Sx=10.0M|
|1419408008\t0R1,Dn=056D,Dm=067D,Dx=080D,Sn=7.1M,Sm=8.9M,Sx=10.0M|
|1419408009\t0R1,Dn=056D,Dm=065D,Dx=080D,Sn=7.1M,Sm=9.1M,Sx=10.0M|
|1419408010\t0R1,Dn=056D,Dm=065D,Dx=080D,Sn=8.5M,Sm=9.3M,Sx=10

In [9]:
sensor.columns

['value']

In [10]:
split_col = pyspark.sql.functions.split(sensor['value'], '\t')

sensor = sensor.withColumn('TIMESTAMP', split_col.getItem(0))
sensor = sensor.withColumn('SENSOR_VALUES', split_col.getItem(1))

sensor.show(truncate=False)

+----------------------------------------------------------------+----------+----------------------------------------------------+
|value                                                           |TIMESTAMP |SENSOR_VALUES                                       |
+----------------------------------------------------------------+----------+----------------------------------------------------+
|1419408000\t0R1,Dn=038D,Dm=079D,Dx=120D,Sn=2.8M,Sm=6.0M,Sx=8.1M |1419408000|0R1,Dn=038D,Dm=079D,Dx=120D,Sn=2.8M,Sm=6.0M,Sx=8.1M |
|1419408001\t0R1,Dn=038D,Dm=074D,Dx=120D,Sn=2.8M,Sm=6.2M,Sx=8.8M |1419408001|0R1,Dn=038D,Dm=074D,Dx=120D,Sn=2.8M,Sm=6.2M,Sx=8.8M |
|1419408002\t0R1,Dn=038D,Dm=071D,Dx=120D,Sn=2.8M,Sm=6.5M,Sx=9.2M |1419408002|0R1,Dn=038D,Dm=071D,Dx=120D,Sn=2.8M,Sm=6.5M,Sx=9.2M |
|1419408003\t0R1,Dn=038D,Dm=067D,Dx=120D,Sn=2.8M,Sm=6.8M,Sx=9.5M |1419408003|0R1,Dn=038D,Dm=067D,Dx=120D,Sn=2.8M,Sm=6.8M,Sx=9.5M |
|1419408004\t0R1,Dn=038D,Dm=062D,Dx=081D,Sn=2.8M,Sm=7.2M,Sx=10.0M|1419408004|0R1,Dn

In [11]:
split_col2 = pyspark.sql.functions.split(sensor['SENSOR_VALUES'], ',')

i = 0
while i <=17:
  sensor = sensor.withColumn('col_{}'.format(i), split_col2.getItem(i))
  i += 1

sensor.show()

+--------------------+----------+--------------------+-----+--------+--------+---------+-------+-------+--------+-----+-----+-----+------+------+------+------+------+------+------+------+
|               value| TIMESTAMP|       SENSOR_VALUES|col_0|   col_1|   col_2|    col_3|  col_4|  col_5|   col_6|col_7|col_8|col_9|col_10|col_11|col_12|col_13|col_14|col_15|col_16|col_17|
+--------------------+----------+--------------------+-----+--------+--------+---------+-------+-------+--------+-----+-----+-----+------+------+------+------+------+------+------+------+
|1419408000\t0R1,D...|1419408000|0R1,Dn=038D,Dm=07...|  0R1| Dn=038D| Dm=079D|  Dx=120D|Sn=2.8M|Sm=6.0M| Sx=8.1M| NULL| NULL| NULL|  NULL|  NULL|  NULL|  NULL|  NULL|  NULL|  NULL|  NULL|
|1419408001\t0R1,D...|1419408001|0R1,Dn=038D,Dm=07...|  0R1| Dn=038D| Dm=074D|  Dx=120D|Sn=2.8M|Sm=6.2M| Sx=8.8M| NULL| NULL| NULL|  NULL|  NULL|  NULL|  NULL|  NULL|  NULL|  NULL|  NULL|
|1419408002\t0R1,D...|1419408002|0R1,Dn=038D,Dm=07...|  0R1|

In [12]:
sensor_clean = sensor.drop('value', 'SENSOR_VALUES')
sensor_clean.show()

+----------+-----+--------+--------+---------+-------+-------+--------+-----+-----+-----+------+------+------+------+------+------+------+------+
| TIMESTAMP|col_0|   col_1|   col_2|    col_3|  col_4|  col_5|   col_6|col_7|col_8|col_9|col_10|col_11|col_12|col_13|col_14|col_15|col_16|col_17|
+----------+-----+--------+--------+---------+-------+-------+--------+-----+-----+-----+------+------+------+------+------+------+------+------+
|1419408000|  0R1| Dn=038D| Dm=079D|  Dx=120D|Sn=2.8M|Sm=6.0M| Sx=8.1M| NULL| NULL| NULL|  NULL|  NULL|  NULL|  NULL|  NULL|  NULL|  NULL|  NULL|
|1419408001|  0R1| Dn=038D| Dm=074D|  Dx=120D|Sn=2.8M|Sm=6.2M| Sx=8.8M| NULL| NULL| NULL|  NULL|  NULL|  NULL|  NULL|  NULL|  NULL|  NULL|  NULL|
|1419408002|  0R1| Dn=038D| Dm=071D|  Dx=120D|Sn=2.8M|Sm=6.5M| Sx=9.2M| NULL| NULL| NULL|  NULL|  NULL|  NULL|  NULL|  NULL|  NULL|  NULL|  NULL|
|1419408003|  0R1| Dn=038D| Dm=067D|  Dx=120D|Sn=2.8M|Sm=6.8M| Sx=9.5M| NULL| NULL| NULL|  NULL|  NULL|  NULL|  NULL|  NULL|

In [13]:
sensor_clean.createOrReplaceGlobalTempView("sensor")

In [14]:
zeroR1 = spark.sql("SELECT TIMESTAMP, col_0, col_1, col_2, col_3, col_4, col_5, col_6  FROM global_temp.sensor WHERE col_0 = '0R1'")

In [15]:
zeroR1.show(5)

+----------+-----+-------+-------+-------+-------+-------+--------+
| TIMESTAMP|col_0|  col_1|  col_2|  col_3|  col_4|  col_5|   col_6|
+----------+-----+-------+-------+-------+-------+-------+--------+
|1419408000|  0R1|Dn=038D|Dm=079D|Dx=120D|Sn=2.8M|Sm=6.0M| Sx=8.1M|
|1419408001|  0R1|Dn=038D|Dm=074D|Dx=120D|Sn=2.8M|Sm=6.2M| Sx=8.8M|
|1419408002|  0R1|Dn=038D|Dm=071D|Dx=120D|Sn=2.8M|Sm=6.5M| Sx=9.2M|
|1419408003|  0R1|Dn=038D|Dm=067D|Dx=120D|Sn=2.8M|Sm=6.8M| Sx=9.5M|
|1419408004|  0R1|Dn=038D|Dm=062D|Dx=081D|Sn=2.8M|Sm=7.2M|Sx=10.0M|
+----------+-----+-------+-------+-------+-------+-------+--------+
only showing top 5 rows



In [16]:
zeroR0 = spark.sql("SELECT * FROM global_temp.sensor WHERE col_0 = '0R0'")
zeroR0.show()

+----------+-----+-------+-------+-------+-------+--------+--------+--------+--------+---------+---------+---------+-------+-------+------+-------+--------+---------+
| TIMESTAMP|col_0|  col_1|  col_2|  col_3|  col_4|   col_5|   col_6|   col_7|   col_8|    col_9|   col_10|   col_11| col_12| col_13|col_14| col_15|  col_16|   col_17|
+----------+-----+-------+-------+-------+-------+--------+--------+--------+--------+---------+---------+---------+-------+-------+------+-------+--------+---------+
|1419408024|  0R0|Dn=057D|Dm=064D|Dx=069D|Sn=8.8M| Sm=9.6M|Sx=10.3M|Ta=13.9C|Ua=28.5P|Pa=889.9H|Rc=76.74M|Rd=34084s|Ri=0.0M|Hc=0.0M| Hd=0s|Hi=0.0M|Vs=25.6V|Vr=3.516V|
|1419408084|  0R0|Dn=056D|Dm=062D|Dx=066D|Sn=8.4M| Sm=8.8M| Sx=9.3M|Ta=13.9C|Ua=28.0P|Pa=889.8H|Rc=76.74M|Rd=34084s|Ri=0.0M|Hc=0.0M| Hd=0s|Hi=0.0M|Vs=25.5V|Vr=3.518V|
|1419408144|  0R0|Dn=058D|Dm=060D|Dx=065D|Sn=8.4M| Sm=9.1M|Sx=10.2M|Ta=14.0C|Ua=27.6P|Pa=889.8H|Rc=76.74M|Rd=34084s|Ri=0.0M|Hc=0.0M| Hd=0s|Hi=0.0M|Vs=25.6V|Vr=3.516V

In [17]:
zeroR2 =  spark.sql("SELECT TIMESTAMP, col_0, col_1, col_2, col_3  FROM global_temp.sensor WHERE col_0 = '0R2'")
zeroR2.show()

+----------+-----+--------+--------+---------+
| TIMESTAMP|col_0|   col_1|   col_2|    col_3|
+----------+-----+--------+--------+---------+
|1419408006|  0R2|Ta=13.9C|Ua=28.5P|Pa=889.9H|
|1419408016|  0R2|Ta=13.9C|Ua=28.5P|Pa=889.9H|
|1419408026|  0R2|Ta=13.9C|Ua=28.4P|Pa=889.9H|
|1419408036|  0R2|Ta=13.9C|Ua=28.3P|Pa=889.7H|
|1419408046|  0R2|Ta=13.9C|Ua=28.3P|Pa=889.9H|
|1419408056|  0R2|Ta=13.9C|Ua=28.0P|Pa=889.9H|
|1419408066|  0R2|Ta=13.9C|Ua=28.0P|Pa=889.9H|
|1419408076|  0R2|Ta=13.9C|Ua=28.0P|Pa=889.8H|
|1419408086|  0R2|Ta=13.9C|Ua=28.0P|Pa=889.9H|
|1419408096|  0R2|Ta=14.0C|Ua=27.8P|Pa=889.9H|
|1419408106|  0R2|Ta=14.0C|Ua=27.8P|Pa=889.9H|
|1419408116|  0R2|Ta=14.0C|Ua=27.9P|Pa=889.9H|
|1419408126|  0R2|Ta=14.0C|Ua=27.8P|Pa=889.9H|
|1419408136|  0R2|Ta=14.0C|Ua=27.6P|Pa=889.8H|
|1419408146|  0R2|Ta=14.1C|Ua=27.4P|Pa=889.8H|
|1419408156|  0R2|Ta=14.1C|Ua=27.5P|Pa=889.8H|
|1419408166|  0R2|Ta=14.1C|Ua=27.4P|Pa=889.7H|
|1419408176|  0R2|Ta=14.1C|Ua=27.5P|Pa=889.9H|
|1419408186| 

In [18]:
zeroR5 =  spark.sql("SELECT TIMESTAMP, col_0, col_1, col_2, col_3, col_4  FROM global_temp.sensor WHERE col_0 = '0R5'")
zeroR5.show()

+----------+-----+--------+-------+--------+---------+
| TIMESTAMP|col_0|   col_1|  col_2|   col_3|    col_4|
+----------+-----+--------+-------+--------+---------+
|1419408023|  0R5|Th=13.1C|Vh=0.0#|Vs=25.6V|Vr=3.516V|
|1419408083|  0R5|Th=13.1C|Vh=0.0#|Vs=25.5V|Vr=3.518V|
|1419408143|  0R5|Th=12.9C|Vh=0.0#|Vs=25.6V|Vr=3.516V|
|1419408203|  0R5|Th=13.3C|Vh=0.0#|Vs=25.6V|Vr=3.516V|
|1419408263|  0R5|Th=13.1C|Vh=0.0#|Vs=25.6V|Vr=3.518V|
|1419408323|  0R5|Th=12.9C|Vh=0.0#|Vs=25.6V|Vr=3.516V|
|1419408383|  0R5|Th=13.3C|Vh=0.0#|Vs=25.7V|Vr=3.515V|
|1419408443|  0R5|Th=13.1C|Vh=0.0#|Vs=25.5V|Vr=3.516V|
|1419408503|  0R5|Th=13.3C|Vh=0.0#|Vs=25.5V|Vr=3.516V|
|1419408563|  0R5|Th=13.3C|Vh=0.0#|Vs=25.6V|Vr=3.516V|
|1419408623|  0R5|Th=13.3C|Vh=0.0#|Vs=25.6V|Vr=3.518V|
|1419408683|  0R5|Th=13.3C|Vh=0.0#|Vs=25.6V|Vr=3.516V|
|1419408743|  0R5|Th=13.3C|Vh=0.0#|Vs=25.6V|Vr=3.515V|
|1419408803|  0R5|Th=13.4C|Vh=0.0#|Vs=25.6V|Vr=3.519V|
|1419408863|  0R5|Th=13.3C|Vh=0.0#|Vs=25.6V|Vr=3.516V|
|141940892

How often (in seconds) do the R5 measurements occur?      
30   
60   
50   
40   

In [19]:
zeroR5.createOrReplaceGlobalTempView("zeroR5_view")

spark.sql("SELECT from_unixtime(TIMESTAMP, 'yyyy-MM-dd HH:mm:ss') as TIME FROM global_temp.zeroR5_view").show()


+-------------------+
|               TIME|
+-------------------+
|2014-12-24 08:00:23|
|2014-12-24 08:01:23|
|2014-12-24 08:02:23|
|2014-12-24 08:03:23|
|2014-12-24 08:04:23|
|2014-12-24 08:05:23|
|2014-12-24 08:06:23|
|2014-12-24 08:07:23|
|2014-12-24 08:08:23|
|2014-12-24 08:09:23|
|2014-12-24 08:10:23|
|2014-12-24 08:11:23|
|2014-12-24 08:12:23|
|2014-12-24 08:13:23|
|2014-12-24 08:14:23|
|2014-12-24 08:15:23|
|2014-12-24 08:16:23|
|2014-12-24 08:17:23|
|2014-12-24 08:18:23|
|2014-12-24 08:19:23|
+-------------------+
only showing top 20 rows



## IMAGE ANALYSIS

In [20]:
cd /content/drive/MyDrive/Colab Notebooks/BigData_week2

/content/drive/MyDrive/Colab Notebooks/BigData_week2


In [21]:
%%bash
python dimensions.py Australia.jpg


size = 5250 columns x 4320 rows
mode = RGB 3x8-bit pixels, true colour


In [22]:
%%bash
python pixel.py Australia.jpg 0 0

(11, 10, 50)


Questions 6 and 7 pertain to the video lecture "Exploring the Array Data Model of an Image")   

What is the (Red, Green, Blue) pixel value for location 500, 2000?   
(134,  145, 46)   
(163, 118, 79)   
(50, 156, 182)   
(100, 123, 149)   

In [23]:
%%bash
python pixel.py Australia.jpg 500 2000

(163, 118, 79)
